In [1]:
import pandas as pd

In [4]:
df_games = pd.read_parquet('../Datasets/steam_games_preprocessed.parquet')
df_user_items = pd.read_parquet('../Datasets/users_items_preprocessed.parquet')

In [5]:
df_games = df_games.rename(columns={'title': 'item_name'})
df_merged = pd.merge(df_user_items, df_games[['item_name', 'release_date', 'genres']], on='item_name')

In [6]:
df_merged.head()

,item_name,playtime_forever,user_id,items_count,release_date,genres
0,Counter-Strike,6,76561197970982479,277,2000-11-01,Action
1,Team Fortress Classic,0,76561197970982479,277,1999-04-01,Action
2,Day of Defeat,7,76561197970982479,277,2003-05-01,Action
3,Deathmatch Classic,0,76561197970982479,277,2001-06-01,Action
4,Half-Life: Opposing Force,0,76561197970982479,277,1999-11-01,Action


In [7]:
df_merged.to_parquet('../Datasets/userforgenre_endpoint.parquet', compression='snappy', index=False)

In [10]:
def UserForGenre(genero: str):
    # Cargar el dataset combinado
    df = pd.read_parquet('../Datasets/userforgenre_endpoint.parquet')
    
    # Filtrar por género
    df = df[df['genres'].str.contains(genero, case=False, na=False)]
    
    # Agrupar por usuario y sumar las horas jugadas
    user_hours = df.groupby('user_id')['playtime_forever'].sum().reset_index()
    
    # Encontrar el usuario con más horas jugadas
    top_user = user_hours.loc[user_hours['playtime_forever'].idxmax()]
    top_user_id = top_user['user_id']
    
    # Filtrar por el usuario con más horas jugadas
    top_user_df = df[df['user_id'] == top_user_id]
    
    # Agrupar las horas jugadas por año de lanzamiento
    top_user_df['year'] = top_user_df['release_date'].dt.year
    hours_by_year = top_user_df.groupby('year')['playtime_forever'].sum().reset_index()
    
    # Formatear el resultado
    hours_by_year_list = hours_by_year.to_dict(orient='records')
    
    return {
        f"Usuario con más horas jugadas para el género {genero}": top_user_id,
        "Horas jugadas": hours_by_year_list
    }


In [11]:
UserForGenre('Action')

C:\Users\santo\AppData\Local\Temp\ipykernel_13168\1371220967.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_user_df['year'] = top_user_df['release_date'].dt.year


{'Usuario con más horas jugadas para Género Action': 'REBAS_AS_F-T',
 'Horas jugadas': [{'year': 1996, 'playtime_forever': 0},
  {'year': 1998, 'playtime_forever': 1},
  {'year': 1999, 'playtime_forever': 224},
  {'year': 2000, 'playtime_forever': 0},
  {'year': 2001, 'playtime_forever': 11},
  {'year': 2002, 'playtime_forever': 1},
  {'year': 2003, 'playtime_forever': 1863},
  {'year': 2004, 'playtime_forever': 2115},
  {'year': 2005, 'playtime_forever': 3293},
  {'year': 2006, 'playtime_forever': 138},
  {'year': 2007, 'playtime_forever': 375},
  {'year': 2008, 'playtime_forever': 2573},
  {'year': 2009, 'playtime_forever': 7926},
  {'year': 2010, 'playtime_forever': 34522},
  {'year': 2011, 'playtime_forever': 45249},
  {'year': 2012, 'playtime_forever': 91435},
  {'year': 2013, 'playtime_forever': 123592},
  {'year': 2014, 'playtime_forever': 191272},
  {'year': 2015, 'playtime_forever': 268357},
  {'year': 2016, 'playtime_forever': 283906},
  {'year': 2017, 'playtime_forever': 199